In [2]:
# Check if the code is running in Colab -- copy/paste this code to your project step 3
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  RunningInColab = True
else:
  print('Not running on CoLab')
  RunningInColab = False

Running on CoLab


In [4]:
# PySpark Pearson correlation function for distributed data processing

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sqrt
from pyspark.sql.types import StructType, StructField, FloatType, IntegerType
from scipy.stats import pearsonr
from itertools import combinations

# Example dataset (replace with your data from GCS)
data = [
    ("Melissa", [4.0, 5.0, 3.8, 4.7]),
    ("Veronica", [3.0, 4.5, 3.2, 4.2]),
    ("Bob", [1.0, 2.0, 3.2, 4.5])
]
# Sample data schema
schema = ["user_name", "rating_list"]

# Create a DataFrame
if RunningInColab == True:
  spark = SparkSession.builder.master("local[*]").getOrCreate()  # we create a spark session if in Google Colab
df = spark.createDataFrame(data, schema)

# Collect the ratings as a dictionary
ratings_dict = df.rdd.map(lambda row: (row["user_name"], row["rating_list"])).collectAsMap()
print(type(ratings_dict))


# Generate all pairs of users (user_name strings)
user_pairs = list(combinations(ratings_dict.keys(), 2))
print(user_pairs)


# Task 1: Your Code Starts Here to compute correlations for all pairs

# Hint: For each pair, you will obtain the ratings for two users from the dictionary,
# and then compute correlation for the two users using Pearson correlation

correlation_results = {}

for user1, user2 in user_pairs:
    ratings1 = ratings_dict[user1]
    ratings2 = ratings_dict[user2]

    # Compute Pearson correlation
    correlation, p_value = pearsonr(ratings1, ratings2)

    # Store results
    correlation_results[(user1, user2)] = (correlation, p_value)
# Your Code Ends Here for the computation



# Task 2: Your Code Starts Here to print results
print("User Pair Correlations:\n")
for (user1, user2), (correlation, p_value) in correlation_results.items():
    print(f"{user1} & {user2}: Pearson Correlation = {correlation:.4f}, p-value = {p_value:.4f}")
# Your Code Ends Here to print results


<class 'dict'>
[('Melissa', 'Veronica'), ('Melissa', 'Bob'), ('Veronica', 'Bob')]
User Pair Correlations:

Melissa & Veronica: Pearson Correlation = 0.9663, p-value = 0.0337
Melissa & Bob: Pearson Correlation = 0.1930, p-value = 0.8070
Veronica & Bob: Pearson Correlation = 0.3836, p-value = 0.6164
